In [3]:
#| echo: false
from bokeh.plotting import figure
from bokeh.models import Range1d, Span
from bokeh.layouts import gridplot
import bokeh.palettes
import numpy as np
import numpy as np
palette = ["#f3c623", "#e84c3d", "#3d97e8", "#3dc8e8", "#6dc8e8", "#8e7fe5", "#f54c4f", "#4c4ff5", "#4cf5b7", "#f5b74c", "#b74cf5", "#4cf54c", "#f54c9c", "#9c4cf5", "#4cf59c", "#f59c4c", "#9c4c8f", "#4c8f9c", "#8f4c9c", "#9c8f4c"]

def plot_format(plot, xlabel, ylabel, location, size, titlesize, labelsize):
    # x axis format
    plot.xaxis.axis_label = xlabel
    plot.xaxis.axis_label_text_font_style = 'bold'
    plot.xaxis.major_label_text_font_style = "bold"
    plot.xaxis.axis_label_text_font_size = size
    plot.xaxis.major_label_text_font_size = size

    # y axis format
    plot.yaxis.axis_label = ylabel
    plot.yaxis.axis_label_text_font_style = 'bold'
    plot.yaxis.major_label_text_font_style = "bold"
    plot.yaxis.axis_label_text_font_size = size
    plot.yaxis.major_label_text_font_size = size

    # Legend format
    plot.legend.location = location
    plot.legend.click_policy = "hide"
    plot.legend.label_text_font_size = labelsize
    plot.legend.label_text_font_style = 'bold'
    plot.legend.border_line_width = 3
    plot.legend.background_fill_alpha = 0.0
    plot.legend.label_text_color = "#E3F4FF"
    # plot.legend.border_line_color = "navy"
    plot.legend.border_line_alpha = 0.0

    # Title format
    plot.title.text_font_size = titlesize
    plot.background_fill_color = "#0E1117"
    plot.border_fill_color = "#0E1117"

    plot.xgrid.grid_line_color = '#2D3135'
    plot.ygrid.grid_line_color = '#2D3135'
    
    plot.yaxis.major_label_text_color = "#E3F4FF"
    plot.xaxis.major_label_text_color = "#E3F4FF"
    plot.yaxis.axis_label_text_color = "#E3F4FF"
    plot.xaxis.axis_label_text_color = "#E3F4FF"
    plot.title.text_color = "#A6DDFF"
    plot.title.text_font_style = "bold"
    
    plot.legend.click_policy="hide"
    return plot

# <span style="color:#A6DDFF">Problem statement - interleaving algorithm</span>

The sampled optical field on the line detector was previously simulated in notebook: [Optical field simulation](a_Optical_Simulation.ipynb).

Some of the observations were:

1. There are 32 sampling points as the line detector has the same number of pixels. This was simulated using a window integration over a defined optical field defined by: $y = e^{-((x-\mu)/\sigma)^n}$. 

2. The parameters $\mu$ and $\sigma$ determine the lateral shift and widht of the gaussian function respectively. These parameters are related to the incoming light angle from the sample and the material roughness.

3. The $\mu$ parameter was swept to simulate different incoming light angles. With this data, a histogram was then reconstructed and its standard deviation value was calculated.  

4. An oscillating pattern was observed in the histogram's standard deviation (Aq parameter). This is not ideal as this indicated a roughness change, however only the angle of the incoming light was being swept.

5. Hence, a new parameter that is constant as a function of $\mu$ has to be found. 

6. It is proposed to implement an interleaving algorithm to increase the density of sampling points as shown in @fig-1



::: {#fig-1}

![](images/b/Fig_1.png)

a. Generate gaussians with different $\mu$ values. b. Calculate a window integration and re-center sampling points. c. Interleave points to increase sampling density. d. Interpolate to get the underlying optical field.
:::

## a. Generate gaussian values with different $\mu$ values